# Tensor renormalization group algorithm

In [ ]:
import numpy as np
import scipy.integrate as integrate

References:  
- H.-J. Liao, J.-G. Liu, L. Wang, T. Xiang:  
  Differentiable programming tensor networks  
  Phys. Rev. X 9, 031041 (2019) (arXiv:1903.09650)  
  https://github.com/wangleiphy/tensorgrad
- M. Levin, C. P. Nave:  
  Tensor renormalization group approach to two-dimensional classical lattice models  
  Phys. Rev. Lett. 99, 120601 (2007) (arXiv:cond-mat/0611687)

In [ ]:
def renormalize(T, Dmax, epsilon):
    """
    Perform a coarse-graining step of the tensor renormalization group (TRG) algorithm.
    """
    # TODO: implement this function

In [ ]:
def trg(T, Dmax, niter, epsilon=1e-15):
    """
    Run the tensor renormalization group (TRG) algorithm, and
    return the logarithm of the partition function (divided by the number of lattice sites).
    """
    logZ = 0
    for n in range(niter):
        print("iteration", n + 1)
        maxval = np.max(abs(T))
        T = T/maxval
        # the power of 2 is the number of the tensors at the current lattice level
        logZ += 2**(niter-n) * np.log(maxval)
        T = renormalize(T, Dmax, epsilon)
    # final trace
    s = T.shape
    logZ += np.log(np.trace(np.reshape(T, (s[0]*s[1], s[2]*s[3]))))
    return logZ / 2**niter

In [ ]:
def construct_ising_tensor(β):
    """
    Construct tensor for evaluating the partition function of the classical Ising model.
    """
    λ = [2*np.cosh(β), 2*np.sinh(β)]
    return np.array(
        [[[[0.5 * np.sqrt(λ[u]*λ[l]*λ[d]*λ[r]) if (u+l+d+r) % 2 == 0 else 0.0
            for r in range(2)]
            for d in range(2)]
            for l in range(2)]
            for u in range(2)])

In [ ]:
def compute_ising_logZ(β, Dmax=12, niter=14):
    """
    Compute the logarithm of the partition function
    of the classical Ising model using the TRG algorithm.
    """
    T = construct_ising_tensor(β)
    return trg(T, Dmax, niter)

In [ ]:
# critical inverse temperature
βc = np.log(1 + np.sqrt(2)) / 2
print("βc:", βc)
# inverse temperature used for the simulation
β = βc - 0.05

In [ ]:
logZ = compute_ising_logZ(β)
logZ

In [ ]:
# analytic reference solution
K = 2*np.sinh(2*β)/np.cosh(2*β)**2
Fref = np.log(2*np.cosh(2*β)) + 1/np.pi*integrate.quad(
    lambda w: np.log(0.5*(1 + np.sqrt(1 - (K*np.sin(w))**2))), 0, np.pi/2)[0]
Fref

In [ ]:
# relative error
abs(logZ - Fref) / Fref